In [ ]:
import os
import numpy as np
import pandas as pd
from glob import glob
import cv2
import torch
from torch.utils import data
from PIL import Image
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import shutil
import utilss
import transforms as T
from engine import train_one_epoch, evaluate
import torch.nn as nn
from IPython.display import clear_output
import pickle

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
#Load saved model
model = torch.load(r'train1000.pkl')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = nn.DataParallel(model)
model.to(device)
print("Model loaded!")

In [ ]:
test_images = glob("test/*.jpg")

In [ ]:
def showbbox(model, img, base_name):

    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])
    b = prediction[0]['boxes']
    s = prediction[0]['scores']
    
    #apply Non-maximum suppression:
    keep = torchvision.ops.nms(b,s,0.1)
        
    img = img.permute(1,2,0)  # C,H,W_H,W,C, for drawing
    img = (img * 255).byte().data.cpu()  # * 255, float to 0-255
    img = np.array(img)  # tensor → ndarray
    #convert np array img to right format.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    classes = { 1: "Ahead_only",
                2: "Beware_of_ice_snow",
                3: "Bicycles_crossing",
                4: "Bumpy_road",
                5: "Children_crossing",
                6: "Dangerous_curve_to_the_left",
                7: "Dangerous_curve_to_the_right",
                8: "Double_curve",
                9: "End_of_all_speed_and_passing_limits",
                10: "End_of_no_passing",
                11: "End_of_no_passing_by_vehicles_over_3_5_metric_tons",
                12: "End_of_speed_limit_80_km_h",
                13: "General_caution",
                14: "Go_straight_or_left",
                15: "Go_straight_or_right",
                16: "Keep_left",
                17: "Keep_right",
                18: "No_entry",
                19: "No_passing",
                20: "No_passing_for_vehicles_over_3_5_metric_tons",
                21: "No_vehicles",
                22: "Pedestrians",
                23: "Priority_road",
                24: "Right_of_way_at_the_next_intersection",
                25: "Road_narrows_on_the_right",
                26: "Road_work",
                27: "Roundabout_mandatory",
                28: "Slippery_road",
                29: "Speed_limit_100_km_h",
                30: "Speed_limit_120_km_h",
                31: "Speed_limit_20_km_h",
                32: "Speed_limit_30_km_h",
                33: "Speed_limit_50_km_h",
                34: "Speed_limit_60_km_h",
                35: "Speed_limit_70_km_h",
                36: "Speed_limit_80_km_h",
                37: "Stop",
                38: "Traffic_signals",
                39: "Turn_left_ahead",
                40: "Turn_right_ahead",
                41: "Vehicles_over_3_5_metric_tons_prohibited",
                42: "Wild_animals_crossing",
                43: "Yield"}

    for k in range(len(keep)):

        xmin = round(prediction[0]['boxes'][k][0].item())
        ymin = round(prediction[0]['boxes'][k][1].item())
        xmax = round(prediction[0]['boxes'][k][2].item())
        ymax = round(prediction[0]['boxes'][k][3].item())
        
        label = prediction[0]['labels'][k].item()
        print(label)
        colors = np.random.uniform(0, 255, size = (43, 3))
        if label in classes:

            pt1 = (xmin, ymin)
            pt2 = (xmax, ymax)
            score = prediction[0]['scores'][k].item()
            color = list(colors[label - 1])
            if score >= 0.6:
                cv2.rectangle(img, pt1, pt2, color, thickness = 2)
                cv2.putText(img, classes[label] + "-" + str(round(score, 2)), (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color,
                        thickness = 2)

    cv2.imwrite(f'results/{base_name}.png', img)

print("Function Loaded")

In [ ]:
for image_path in test_images:
    
    img = cv2.imread(image_path)
    #convert from BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #convert the numpy array to PIL Image
    img = Image.fromarray(img)
    img_tensor = transforms.ToTensor()(img)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    showbbox(model, img_tensor, base_name)